# Predicting growth of sourdough bread market size based on global trends
### Data Engineering Capstone Project

#### Project Summary
This project aims to identify trends in a growing community of sourdough bread enthusiasts based on global trends. The company, Sourdough Queen, has seen a rise in sales due to COVID-19 and stay home policy and wishes to analyze past trends in global sourdough community based on newsworthy events. They hope to use this information predict spikes in sales volume in future years. As a result, Sourdough Queen will be able to prepare adequate inventory as well as create highly engaging and relevant social media campaigns to drive more visitors to their website and e-commerce store.

The objective of the current notebook is to create a data pipeline that will provide the neccessary data for a data scientist at Sourdough Queen to build a predictive model for anticipating growith in the sourdough market based on global events.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd

# Google Trends API
#from pytrends.request import TrendReq

#### Using the Google Trends API

In [9]:
#!pip install pytrends

In [ ]:
#pytrend = TrendReq()

In [ ]:
# Read in the data here
#kw_list = ["insurrection", "sourdough", "quarantine", "lockdown", "outbreak"] #would need 4000 keywords
#pytrend.build_payload(kw_list, cat=0, timeframe='2018-10-01 2018-12-31', geo='', gprop='') #need to increase to make 1 million rows, but can't use API at once to get this

In [ ]:
# Interest by Region
#search_history_by_region = pytrend.interest_by_region()
#search_history_by_region.head(10)

In [ ]:
#p = search_history_by_region.reset_index().plot(x='geoName', y='outbreak', figsize=(120, 10), kind ='bar')

In [ ]:
#search_history_by_region.info()

In [ ]:
#Get data for the last 17 years
#hourly_data = pytrend.get_historical_interest(kw_list, year_start=2018, month_start=10, day_start=1, hour_start=0, year_end=2018, month_end=12, day_end=31, hour_end=0, cat=0, geo='', gprop='', sleep=0)

In [ ]:
#hourly_data.info()
#approx 10,000 lines * 250 countries = 2.5 million

In [ ]:
#hourly_data.tail()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
##### 1.1 Million News Headlines
This contains data of news headlines published over a period of 17 years. Sourced from the reputable Australian news source ABC (Australian Broadcasting Corporation) 
This news dataset is a summarised historical record of noteworthy events in the globe from early-2003 to end-2019 with a more granular focus on Australia. 
This includes the entire corpus of articles published by the ABC website in the given time range. With a volume of two hundred articles each day and a good focus on international news, we can be fairly certain that every event of significance has been captured here. 
Digging into the keywords, one can see all the important episodes shaping the last decade and how they evolved over time. For example: financial crisis, iraq war, multiple elections, ecological disasters, terrorism, famous people, local crimes etc.

In [5]:
# Read in the data here
news_df = pd.DataFrame(pd.read_csv("data/abcnews-date-text.csv"))

##### 1.2 Global trends by hour
This data includes interest over time for 'sourdough' over the past 5 years (`data/multiTimeline.csv`) as well as interest by countries in the world (`data/geoMap.csv`). Search interest is calculated as a score and interpreted as search interest relative to the highest value for the given region and time. A value of 100 is the peak popularity for the term. A value of 50 means that the term is half as popular. A score of 0 means that there was not enough data for this term.

In [8]:
# Read in the data here
interest_by_date_df = pd.DataFrame(pd.read_csv("data/multiTimeline.csv"))
interest_by_country_df = pd.DataFrame(pd.read_csv("data/geoMap.csv"))

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

##### 2.1 Explore trends data

In [11]:
interest_by_date_df.head()

,date,search_interest_sourdough_worldwide
0,2015-01-01T00:00:00Z,7
1,2015-02-01T00:00:00Z,7
2,2015-03-01T00:00:00Z,6
3,2015-03-31T23:00:00Z,6
4,2015-04-30T23:00:00Z,6


In [13]:
interest_by_country_df.head()

,country,sourdough: (01/01/2015 - 31/12/2020),relative_search_interest
0,Guernsey,NaN,NaN
1,Jersey,NaN,NaN
2,New Zealand,100.0,100.0
3,Australia,100.0,100.0
4,Iceland,NaN,NaN


In [10]:
interest_by_date_df.describe()

,search_interest_sourdough_worldwide
count,72.000000
mean,14.513889
std,15.256486
min,6.000000
25%,8.000000
50%,11.000000
75%,13.000000
max,100.000000


In [14]:
interest_by_country_df.describe()

,sourdough: (01/01/2015 - 31/12/2020),relative_search_interest
count,39.000000,39.000000
mean,21.205128,21.205128
std,30.824785,30.824785
min,1.000000,1.000000
25%,2.000000,2.000000
50%,7.000000,7.000000
75%,20.500000,20.500000
max,100.000000,100.000000


##### Check for missing values

In [12]:
interest_by_date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 2 columns):
date                                   72 non-null object
search_interest_sourdough_worldwide    72 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [15]:
interest_by_country_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
country                                 250 non-null object
sourdough: (01/01/2015 - 31/12/2020)    39 non-null float64
relative_search_interest                39 non-null float64
dtypes: float64(2), object(1)
memory usage: 5.9+ KB


##### Cleaning Steps
Document steps necessary to clean the data

In [23]:
#convert to datetime
interest_by_date_df['date'] = pd.to_datetime(interest_by_date_df['date'])
#interest_by_date_df.info()

#create new columns for year, month, day, day_of_week
interest_by_date_df['year']= interest_by_date_df['date'].dt.year
interest_by_date_df['month']= interest_by_date_df['date'].dt.month
interest_by_date_df['day']= interest_by_date_df['date'].dt.day
interest_by_date_df['day_of_week']= interest_by_date_df['date'].dt.dayofweek

#view dataframe
interest_by_date_df.tail()

,date,search_interest_sourdough_worldwide,year,month,day,day_of_week
67,2020-07-31 23:00:00,31,2020,7,31,4
68,2020-08-31 23:00:00,26,2020,8,31,0
69,2020-09-30 23:00:00,24,2020,9,30,2
70,2020-10-31 23:00:00,24,2020,10,31,5
71,2020-12-01 00:00:00,25,2020,12,1,1


In [20]:
# Checkout interest by country data set
interest_by_country_df.head(10)

,country,sourdough: (01/01/2015 - 31/12/2020),relative_search_interest
0,Guernsey,NaN,NaN
1,Jersey,NaN,NaN
2,New Zealand,100.0,100.0
3,Australia,100.0,100.0
4,Iceland,NaN,NaN
5,Singapore,82.0,82.0
6,Canada,82.0,82.0
7,Ireland,81.0,81.0
8,United Kingdom,80.0,80.0
9,Bermuda,NaN,NaN


In [23]:
# melt to long form
interest_by_country_df.melt(id_vars=['country'])

,country,variable,value
0,Guernsey,sourdough: (01/01/2015 - 31/12/2020),NaN
1,Jersey,sourdough: (01/01/2015 - 31/12/2020),NaN
2,New Zealand,sourdough: (01/01/2015 - 31/12/2020),100.0
3,Australia,sourdough: (01/01/2015 - 31/12/2020),100.0
4,Iceland,sourdough: (01/01/2015 - 31/12/2020),NaN
5,Singapore,sourdough: (01/01/2015 - 31/12/2020),82.0
6,Canada,sourdough: (01/01/2015 - 31/12/2020),82.0
7,Ireland,sourdough: (01/01/2015 - 31/12/2020),81.0
8,United Kingdom,sourdough: (01/01/2015 - 31/12/2020),80.0
9,Bermuda,sourdough: (01/01/2015 - 31/12/2020),NaN


##### Now we need to: 

##### 1. Create a dataset of dates based on the range of dates in the above data set

##### 2. Join dates with countries to create bigger dataset

In [22]:
#Create dates denominator (every day past 5 years)
dates_df = pd.DataFrame(pd.date_range(start='01/01/2015', end='31/12/2020'))
dates_df.tail(20)

,0
2172,2020-12-12
2173,2020-12-13
2174,2020-12-14
2175,2020-12-15
2176,2020-12-16
2177,2020-12-17
2178,2020-12-18
2179,2020-12-19
2180,2020-12-20
2181,2020-12-21


In [24]:
# combine dates and countries together
country_date = pd.concat([interest_by_country_df, dates_df], axis=1, join="outer")
country_date

,country,sourdough: (01/01/2015 - 31/12/2020),relative_search_interest,0
0,Guernsey,NaN,NaN,2015-01-01
1,Jersey,NaN,NaN,2015-01-02
2,New Zealand,100.0,100.0,2015-01-03
3,Australia,100.0,100.0,2015-01-04
4,Iceland,NaN,NaN,2015-01-05
5,Singapore,82.0,82.0,2015-01-06
6,Canada,82.0,82.0,2015-01-07
7,Ireland,81.0,81.0,2015-01-08
8,United Kingdom,80.0,80.0,2015-01-09
9,Bermuda,NaN,NaN,2015-01-10


In [37]:
#merge country_date with interest_by_date

trends_df = pd.DataFrame(pd.concat([country_date, interest_by_date_df], axis=1, join="inner"))
trends_df.head()

,country,sourdough: (01/01/2015 - 31/12/2020),relative_search_interest,0,date,search_interest_sourdough_worldwide
0,Guernsey,NaN,NaN,2015-01-01,2015-01-01T00:00:00Z,7
1,Jersey,NaN,NaN,2015-01-02,2015-02-01T00:00:00Z,7
2,New Zealand,100.0,100.0,2015-01-03,2015-03-01T00:00:00Z,6
3,Australia,100.0,100.0,2015-01-04,2015-03-31T23:00:00Z,6
4,Iceland,NaN,NaN,2015-01-05,2015-04-30T23:00:00Z,6


In [42]:
#convert to datetime
trends_df['date'] = pd.to_datetime(trends_df['date'])

#create new columns for year, month, day, day_of_week
trends_df['year'] = trends_df['date'].dt.year
trends_df['month'] = trends_df['date'].dt.month
trends_df['day'] = trends_df['date'].dt.day
trends_df['day_of_week'] = trends_df['date'].dt.dayofweek

#view dataframe
trends_df.head()

,country,sourdough: (01/01/2015 - 31/12/2020),relative_search_interest,0,date,search_interest_sourdough_worldwide,year,month,day,day_of_week
0,Guernsey,NaN,NaN,2015-01-01,2015-01-01 00:00:00,7,2015,1,1,3
1,Jersey,NaN,NaN,2015-01-02,2015-02-01 00:00:00,7,2015,2,1,6
2,New Zealand,100.0,100.0,2015-01-03,2015-03-01 00:00:00,6,2015,3,1,6
3,Australia,100.0,100.0,2015-01-04,2015-03-31 23:00:00,6,2015,3,31,1
4,Iceland,NaN,NaN,2015-01-05,2015-04-30 23:00:00,6,2015,4,30,3


In [43]:
#melt to long form by date
trends_df_expanded = trends_df.melt(id_vars=['year', 'month', 'day', 'day_of_week', 'country', 'search_interest_sourdough_worldwide', 'relative_search_interest'])
trends_df_expanded.describe()

,year,month,day,day_of_week,search_interest_sourdough_worldwide,relative_search_interest
count,216.000000,216.000000,216.000000,216.000000,216.000000,75.000000
mean,2017.500000,5.833333,20.750000,3.111111,14.513889,31.880000
std,1.711792,3.294816,14.003405,1.987558,15.185360,33.789475
min,2015.000000,1.000000,1.000000,0.000000,6.000000,4.000000
25%,2016.000000,3.000000,1.000000,1.000000,8.000000,8.000000
50%,2017.500000,5.500000,30.000000,3.000000,11.000000,12.000000
75%,2019.000000,8.250000,31.000000,5.000000,13.000000,60.000000
max,2020.000000,12.000000,31.000000,6.000000,100.000000,100.000000


#### Check for missing values

In [44]:
trends_df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 9 columns):
year                                   216 non-null int64
month                                  216 non-null int64
day                                    216 non-null int64
day_of_week                            216 non-null int64
country                                216 non-null object
search_interest_sourdough_worldwide    216 non-null int64
relative_search_interest               75 non-null float64
variable                               216 non-null object
value                                  169 non-null object
dtypes: float64(1), int64(5), object(3)
memory usage: 15.3+ KB


Good, everything is as expected.

In [45]:
#save final trends data table
trends_df_expanded.to_csv('trends_df_expanded.csv')

##### 2.2 Explore news data

In [16]:
news_df.head()

,publish_date,headline_text
0,2003-02-19,aba decides against community broadcasting lic...
1,2003-02-19,act fire witnesses must be aware of defamation
2,2003-02-19,a g calls for infrastructure protection summit
3,2003-02-19,air nz staff in aust strike for pay rise
4,2003-02-19,air nz strike to affect australian travellers


In [18]:
news_df.describe()

,publish_date,headline_text
count,1186018,1186018
unique,6152,1155838
top,+20120824-01-01,national rural news
freq,384,983


In [19]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186018 entries, 0 to 1186017
Data columns (total 2 columns):
publish_date     1186018 non-null object
headline_text    1186018 non-null object
dtypes: object(2)
memory usage: 18.1+ MB


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model:

We chose a star schema because given the current data needs of this startup, this model will provide performance without additional overhead for their small data science team. We think this is the best option that will provide the best value and flexibility for future expansions to their existing pipeline without adding a heavy meaintenence load. 

The conceptual data model is as follows:
###### **Fact table**
1. **global_trends** - monthly search trends associated with news headlines
    - *global_trends_id, month, day, day_of_week, publish_date, headline_text*

###### **Dimension tables**
1. **news** - news headlines by date
    - *publish_date, headline_text*
2. **trends** - search interest by date and country
    - *year, month, day, day_of_week, search_interest_sourdough_worldwide, relative_serach_interest*
3. **time** - global news headline publish dates broken down into specific time units
    - *publish_date, day, month, year, day_of_week*

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model:

The following steps were used to process the raw data into the tables in our database for further analysis as outlined in `etl.py`:

1. **trends table**
    - create temporary view of the trends table
    - select and insert the year, month, day, day_of_week, search_interest_sourdough_worldwide, relative_serach_interest data 
    - save trends table to S3

2. **news table**
    - create temporary view of the news table
    - select and insert the spublish_date, headline_text data into the news table
    - save news table to S3

3. **time table**
    - create temporary view of the time table
    - create a new column `ts` using `publish_date`
    - convert the `ts` column to datetime
    - calculate the time data as a dataframe
    - insert the day, month, year, and day_of_week data into the time table
    - save time table to S3

4. **global_trends table**
    - create a primary key as `global_trends_id`
    - join news, trends and time tables by month, day and day_of_week to create global_trends table
    - save global_trends table to S3

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [60]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

In [61]:
spark = SparkSession \
.builder \
.appName("Data Quality Check") \
.getOrCreate()

In [62]:
spark.sparkContext.getConf().getAll()

[('spark.driver.host', '58710323a069'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'Data Quality Check'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1611513981628'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '44301')]

##### 4.2.1 Trends table

In [63]:
#path = "data/trends_df_expanded.csv"

In [64]:
trends_data = spark.read.csv(path)

In [65]:
trends_data.createOrReplaceTempView("trends_table")

In [70]:
# check that the loading script is working
spark.sql("SELECT * FROM trends_table LIMIT 5").show()

+----+----+-----+---+-----------+-----------+--------------------+--------------------+--------------------+-----+
| _c0| _c1|  _c2|_c3|        _c4|        _c5|                 _c6|                 _c7|                 _c8|  _c9|
+----+----+-----+---+-----------+-----------+--------------------+--------------------+--------------------+-----+
|null|year|month|day|day_of_week|    country|search_interest_s...|relative_search_i...|            variable|value|
|   0|2015|    1|  1|          3|   Guernsey|                   7|                null|sourdough: (01/01...| null|
|   1|2015|    2|  1|          6|     Jersey|                   7|                null|sourdough: (01/01...| null|
|   2|2015|    3|  1|          6|New Zealand|                   6|               100.0|sourdough: (01/01...|100.0|
|   3|2015|    3| 31|          1|  Australia|                   6|               100.0|sourdough: (01/01...|100.0|
+----+----+-----+---+-----------+-----------+--------------------+--------------

In [71]:
# check that we have all the information we expect
spark.sql("SELECT DISTINCT year FROM trends_table").show()

+----+
| _c1|
+----+
|2016|
|2020|
|2019|
|2017|
|2018|
|year|
|2015|
+----+



In [72]:
# check that we have all the information we expect
spark.sql("SELECT DISTINCT country FROM trends_table").show()

+-------------+
|          _c5|
+-------------+
|        Macao|
|       Sweden|
|       Jersey|
|  Philippines|
|    Singapore|
|     Malaysia|
|     Cambodia|
|     Maldives|
|       Greece|
|     Slovakia|
|      Belgium|
|        Qatar|
|      Albania|
|      Finland|
|     Guernsey|
|United States|
|      Bahamas|
|        China|
|        Malta|
|       Kuwait|
+-------------+
only showing top 20 rows



##### 4.2.2 News Table

In [73]:
#path = "data/abcnews-date-text.csv"

In [75]:
news_data = spark.read.csv(path)

In [79]:
news_data.createOrReplaceTempView("news_table")

In [82]:
# check data type and schema for news table
news_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [80]:
# check news data is loaded properly
spark.sql("SELECT * FROM news_table LIMIT 5").show()

+------------+--------------------+
|         _c0|                 _c1|
+------------+--------------------+
|publish_date|       headline_text|
|  2003-02-19|aba decides again...|
|  2003-02-19|act fire witnesse...|
|  2003-02-19|a g calls for inf...|
|  2003-02-19|air nz staff in a...|
+------------+--------------------+



In [81]:
# check that we have 1 million rows in the news table
news_data.count()

1186019

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.